In [ ]:
import boto3
import configparser
#import matplotlib.pyplot as plt
import pandas as pd
from time import time
import psycopg2

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY=config.get('AWS','key')
SECRET=config.get('AWS','secret')
REGION=config.get('AWS','REGION')

DWH_DB=config.get("DWH","DWH_DB")
DWH_DB_USER=config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD=config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT=config.get("DWH","DWH_PORT")

DWH_ROLE_ARN=config.get("DWH","DWH_ROLE_ARN")
DWH_ENDPOINT=config.get("DWH","DWH_ENDPOINT")


In [ ]:
#Create connection

conn = psycopg2.connect(
    host=DWH_ENDPOINT,
    dbname=DWH_DB,
    user=DWH_DB_USER,
    password=DWH_DB_PASSWORD,
    port=DWH_PORT
)
print("Connection successful!")

cur = conn.cursor()

def execute_query(query, params=None):
    try:
        if params:
            cur.execute(query, params)
        else:
            cur.execute(query)
        conn.commit()
        print("Query executed successfully")
        return cur.fetchall()
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
        return None


In [ ]:
s3 = boto3.resource('s3', region_name=REGION, aws_access_key_id=KEY, aws_secret_access_key=SECRET)
sampleDbBucket=s3.Bucket("udacity-labs")
for obj in sampleDbBucket.objects.filter(Prefix="tickets"):
    print(obj)


In [ ]:
create_ticket_table = """
DROP TABLE IF EXISTS "sporting_event_ticket";

CREATE TABLE "sporting_event_ticket" (
    "id" double precision DEFAULT nextval('sporting_event_ticket_seq') NOT NULL,
    "sporting_event_id" double precision NOT NULL,
    "sport_location_id" double precision NOT NULL,
    "seat_level" numeric(1,0) NOT NULL,
    "seat_section" character varying(15) NOT NULL,
    "seat_row" character varying(10) NOT NULL,
    "seat" character varying(10) NOT NULL,
    "ticketholder_id" double precision,
    "ticket_price" numeric(8,2) NOT NULL
);
"""

result = execute_query(create_ticket_table)
print(result)

In [ ]:
load_partition_data = f"""

    COPY sporting_event_ticket FROM 's3://udacity-labs/tickets/split/part'
    credentials 'aws_iam_role={DWH_ROLE_ARN}' gzip DELIMITER ';' region '{REGION}';
"""

result = execute_query(load_partition_data)
print(result)

In [ ]:
create_full_tickets_table = """
DROP TABLE IF EXISTS "sporting_event_ticket_full";

CREATE TABLE "sporting_event_ticket_full" (
    "id" double precision DEFAULT nextval('sporting_event_ticket_seq') NOT NULL,
    "sporting_event_id" double precision NOT NULL,
    "sport_location_id" double precision NOT NULL,
    "seat_level" numeric(1,0) NOT NULL,
    "seat_section" character varying(15) NOT NULL,
    "seat_row" character varying(10) NOT NULL,
    "seat" character varying(10) NOT NULL,
    "ticketholder_id" double precision,
    "ticket_price" numeric(8,2) NOT NULL
);
"""

result = execute_query(create_full_tickets_table)
print(result)

In [ ]:
load_full_data = f"""
    copy sporting_event_ticket_full from 's3://udacity-labs/tickets/full/full.csv.gz' 
    credentials 'aws_iam_role={DWH_ROLE_ARN}' 
    gzip delimiter ';' region 'us-west-2'
    """
    
result = execute_query(load_full_data)
print(result)